In [51]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization
import tqdm 
import os
import io


In [24]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [25]:
sampling_table =  tf.keras.preprocessing.sequence.make_sampling_table(4096)

print(sampling_table)

[0.00315225 0.00315225 0.00547597 ... 0.60335544 0.60343743 0.60351941]


In [26]:
seed = 39
vocab_size = 4096
num_ns = 4
sequence_length = 25

train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "dataset", batch_size = 1024, seed = seed
)

text_ds = train_ds.map(lambda x, y: x)

vectorize_layer = TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int', 
    output_sequence_length = sequence_length
)

vectorize_layer.adapt(text_ds)

inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:60])




Found 230710 files belonging to 2 classes.
['', '[UNK]', 'the', 'trump', 'to', 'a', 'of', 'and', 'in', 'that', 'his', 'for', 'on', 'he', 'is', 'with', 'donald', 'said', 'as', 'president', 'has', 'was', 'at', 'by', 'it', 'who', 'from', 'be', 'not', 'but', 'an', 'have', 'about', 'would', 'had', 'will', 'campaign', 'this', 'are', 'i', 'full', 'or', 'him', 'text', 'new', 'after', 'they', 'if', 'one', 'more', 'when', 'house', 'republican', 'clinton', 'its', 'been', 'what', 'which', 'their', 'out']


In [27]:

text_vector_ds = text_ds.prefetch(tf.data.AUTOTUNE).map(vectorize_layer).unbatch()

print(inverse_vocab[:60])

['', '[UNK]', 'the', 'trump', 'to', 'a', 'of', 'and', 'in', 'that', 'his', 'for', 'on', 'he', 'is', 'with', 'donald', 'said', 'as', 'president', 'has', 'was', 'at', 'by', 'it', 'who', 'from', 'be', 'not', 'but', 'an', 'have', 'about', 'would', 'had', 'will', 'campaign', 'this', 'are', 'i', 'full', 'or', 'him', 'text', 'new', 'after', 'they', 'if', 'one', 'more', 'when', 'house', 'republican', 'clinton', 'its', 'been', 'what', 'which', 'their', 'out']


In [28]:


sequences = list(text_vector_ds.as_numpy_iterator())

targets, contexts, labels = generate_training_data(
    sequences = sequences, 
    window_size=2, 
    num_ns=num_ns, 
    vocab_size = vocab_size,
    seed = seed
)

BATCH_SIZE = 1024
BUFFER_SIZE = 10000
AUTOTUNE = tf.data.AUTOTUNE

dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset = dataset.cache().prefetch(buffer_size = AUTOTUNE)



100%|██████████| 230710/230710 [06:39<00:00, 576.97it/s]


In [40]:
class Word2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()

        self.target_embedding = Embedding(vocab_size,
        embedding_dim,
        input_length=1,
        name="w2v_embedding"
        )

        self.context_embedding = Embedding(vocab_size,
        embedding_dim,
        input_length=num_ns+1
        )

    def call(self, pair):
         target, context = pair

         if len(target.shape) == 2:
            target = tf.squeeze(target, axis=1)

         word_emb = self.target_embedding(target)

         context_emb = self.context_embedding(context)

         context_emb = tf.squeeze(context_emb, axis = 2)

         dots = tf.einsum('be,bce->bc', word_emb, context_emb)

         return dots
        

In [41]:
embedding_dim = 128
word2Vec = Word2Vec(vocab_size, embedding_dim)
word2Vec.compile(optimizer = 'adam', 
                 loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
                 metrics = ['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

word2Vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
1774/1774 [==============================] - 41s 23ms/step - loss: 1.3977 - accuracy: 0.4147
Epoch 2/20
1774/1774 [==============================] - 34s 19ms/step - loss: 1.2162 - accuracy: 0.5118
Epoch 3/20
1774/1774 [==============================] - 37s 21ms/step - loss: 1.1353 - accuracy: 0.5488
Epoch 4/20
1774/1774 [==============================] - 36s 20ms/step - loss: 1.0805 - accuracy: 0.5728
Epoch 5/20
1774/1774 [==============================] - 39s 22ms/step - loss: 1.0377 - accuracy: 0.5911
Epoch 6/20
1774/1774 [==============================] - 41s 23ms/step - loss: 1.0020 - accuracy: 0.6061
Epoch 7/20
1774/1774 [==============================] - 40s 23ms/step - loss: 0.9715 - accuracy: 0.6190
Epoch 8/20
1774/1774 [==============================] - 38s 22ms/step - loss: 0.9451 - accuracy: 0.6298
Epoch 9/20
1774/1774 [==============================] - 34s 19ms/step - loss: 0.9221 - accuracy: 0.6393
Epoch 10/20
1774/1774 [==============================] - 34s 19m

In [42]:
weights = word2Vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

out_v = io.open('w2vVectors.tsv', 'w', encoding='utf-8')
out_m = io.open('w2Vmetadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [50]:
print(len(word2Vec.get_layer('w2v_embedding').get_weights()[0]))

4096


In [ ]:
np.save('w2vVectors.npy', weights)